In [4]:
from collections import defaultdict
from collections import Counter
from ete3 import PhyloTree
import numpy as np
from tqdm import tqdm
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

import time
import csv
import copy
import pandas as pd
from glob import glob
from itertools import combinations
from ete3 import Tree

Load gene families

In [2]:
spsel=set(['Acapla','Annjap', 'Apojap', 'Astrub', 'Auraur', 'Braflo', 'Bralan', 'Calmil', 'Captel', 'Cioint', 'Dromel', 'Homsap', 'Lepocu', 'Limpol', 'Linana', 'Lotgig', 'Lytvar', 'Molocc', 'Nemvec', 'Parliv', 'Patmin', 'Patyes', 'Ptyfla', 'Sackow', 'Strpur', 'Tricas'])

In [24]:
spsel=set(['Acapla','Annjap', 'Apojap', 'Auraur', 'Braflo', 'Bralan', 'Calmil', 'Captel', 'Cioint', 'Dromel', 'Homsap', 'Lepocu', 'Limpol', 'Linana', 'Lotgig', 'Lytvar', 'Molocc', 'Nemvec', 'Parliv', 'Patmin', 'Patyes', 'Ptyfla', 'Sackow', 'Strpur', 'Tricas'])

In [30]:
Fams,species={},[]
for line in open('orthologous_groups.txt'):
    if line.startswith("#"): continue
    fid,glt=line.strip().split('\t')
    gbs=defaultdict(list)
    for g in glt.split():
        if not len(g.split('_',1))==2: continue
        spc,gid=g.split('_',1)
        if not spc in spsel: continue 
        gbs[spc].append(gid)
    species.extend(gbs.keys())
    if len(gbs.keys())<=1: continue
    Fams[fid]=gbs
        #if i==5: break
species=list(set(species))

In [14]:
print(list(Fams.items())[0])
print(len(Fams))
print(sorted(species))

('OG_1', defaultdict(<class 'list'>, {'Homsap': ['ENSG00000168944'], 'Lytvar': ['L_var_10415-RA'], 'Lepocu': ['ENSLOCG00000008635'], 'Ptyfla': ['pfl_40v0_9_20150316_1g17968'], 'Strpur': ['LOC577882'], 'Nemvec': ['NEMVEDRAFT_v1g116171'], 'Parliv': ['PL00614'], 'Limpol': ['XP_013777632', 'XP_013777633', 'XP_013777634'], 'Cioint': ['ENSCING00000020805'], 'Apojap': ['BSL78_15936', 'BSL78_11218'], 'Sackow': ['Sakowv30036170m'], 'Auraur': ['scaffold7269_g1', 'scaffold5741_g1', 'scaffold723_g2'], 'Linana': ['LANA06268', 'LANA06269'], 'Patmin': ['PMI_004327-RA', 'PMI_016166-RA'], 'Molocc': ['g27598', 'g27599', 'g27601', 'g27600'], 'Tricas': ['TC015682', 'TC015681'], 'Braflo': ['BL074181'], 'Captel': ['CapteG32511'], 'Bralan': ['BL08976'], 'Patyes': ['PY_T15208'], 'Annjap': ['LOC117108548'], 'Lotgig': ['LotgiG190538'], 'Calmil': ['SINCAMG00000005543'], 'Acapla': ['LOC110981048']}))
18758
['Acapla', 'Annjap', 'Apojap', 'Auraur', 'Braflo', 'Bralan', 'Calmil', 'Captel', 'Cioint', 'Dromel', 'Homsap

Creates non-redundant species set

Load species tree

In [10]:
spT=PhyloTree(open('species_wa.tre').read(), format=1)

### Determine phylostrata of  genes 

In [15]:
print(Fams['OG_171'])

defaultdict(<class 'list'>, {'Acapla': ['LOC110984554']})


In [29]:
urchinPhylo=defaultdict(int)
g2f={}
genephylo={}
tor=defaultdict(int)
size=[]
for fid in Fams:
    gbs=Fams[fid]
    spset=[s for s,gbs in gbs.items() if len(gbs)>0]
    if len(spset)==0: continue
    phtyp=spT.get_common_ancestor(spset) if len(spset)>1 else spT.search_nodes(name=spset[0])
    #print(fid,len(phtyp))
    tor[phtyp.name]+=1
    size.append(len(spset))
    for g in gbs['Parliv']:
        urchinPhylo[phtyp.name]+=1
        genephylo[g]=phtyp.name
        g2f[g]=fid
        

print(urchinPhylo,sum(urchinPhylo.values()))

In [31]:
selclades=['Eumetazoa','Bilateria','Deuterostomia','Ambulacraria', 'Echinodermata','Eleutherozoa','Echinozoa','Echinoidea']

In [32]:
Counter(size)

Counter({25: 1865,
         26: 1878,
         24: 1315,
         20: 414,
         23: 914,
         17: 281,
         12: 181,
         11: 176,
         22: 711,
         21: 539,
         19: 326,
         7: 307,
         6: 314,
         15: 247,
         18: 315,
         14: 197,
         3: 1827,
         4: 552,
         8: 231,
         13: 183,
         5: 364,
         9: 217,
         16: 263,
         10: 184,
         2: 5157})

## Extract TFs

In [9]:
hsa_tfs=set([l.strip().split('\t')[1] for l in open('hsa_tfs.txt')])

In [ ]:
print(hsa_tfs)

In [18]:
print(len([]))

0


In [76]:
pltf=[]
ucfn=0
with open('Pliv_tfs.tsv','w') as out:
    for f,gbs in Fams.items():
        hgs=set(gbs.get('Homsap',[]))
        tfhg=hgs.intersection(hsa_tfs)
        if len(tfhg)>=1:
            tfpl=gbs.get('Parliv',[])
            if len(tfpl)>=1:
                for g in tfpl:
                    if not g in PlivNames: ucfn+=1 
                    out.write("{}\n".format('\t'.join([f,g,PlivNames.get(g,'Unk_{}'.format(ucfn)),','.join(Panther.get(g,'-')),gPos.get(g,''),','.join([HsaNames[g] for g in tfhg])])))
                        
            #print(f,tfhg,[HsaNames[g] for g in tfhg],tfpl,[PlivNames.get(g,'') for g in tfpl])
    
    

## Testing family expansions

In [8]:
#only testing species expansion
def fishExp(gbs,tbs):
    cbs=dict((sp,len(gs)) for sp,gs in gbs.items())
    med=np.median(list(cbs.values()))
    #print(cbs,med)
    spval={}
    for sp,st in cbs.items():
        nis=sum(cbs.values())-st
        nit=sum(tbs.values())-tbs[sp]-sum(cbs.values())+st
        odds,pval=stats.fisher_exact([[st, tbs[sp]-st], [nis, nit]],alternative='greater')
        nmed=np.median([cbs[ssp] for ssp in cbs if not ssp==sp])
        tmed=np.median([tbs[ssp] for ssp in tbs if not ssp==sp])
        m_odds,m_pval=stats.fisher_exact([[st, tbs[sp]-st], [nmed, tmed]],alternative='greater')
        #print(sp,pval,[st, tbs[sp]-st], [nis, nit],m_pval,[st, tbs[sp]-st], [nmed, tmed])
        spval[sp]=(cbs[sp],m_pval)

    return spval

calculate number of genes per species

In [9]:
tbs=defaultdict(int)
for fid,gbs in Fams.items():
    for sp in gbs:
        tbs[sp]+=len(gbs[sp])

In [10]:
toosmall=0
expRes=[]
for fid,gbs in tqdm(Fams.items()):
    cbs=dict((sp,len(gs)) for sp,gs in gbs.items())
    if len(cbs.keys())<3 or sum(cbs.values())<5:
        toosmall+=1
        continue
    try:
        spval=fishExp(gbs,tbs)
        expRes.append((fid,spval))
    except:
        print(fid,cbs)

100%|██████████| 18958/18958 [01:51<00:00, 170.58it/s]


In [11]:
print(len(Fams),len(expRes),toosmall)

18958 12418 6540


In [12]:
spFam,spPval=defaultdict(list),defaultdict(list)
for fid,spval in expRes:
    for sp in spval:
        ct,pval=spval[sp]
        spFam[sp].append(fid)                     
        spPval[sp].append(pval)

In [13]:
famEnrich=defaultdict(dict)
for sp in spPval:
    #signif_pvals = sidak(spPval[sp], alpha=0.05)
    #before_adj=['True' if p < 0.05  else 'False' for p in spPval[sp] ]
    #1signif_pvals = lsu(np.array(spPval[sp]), q=0.05) #this is benferroni-hochberg
    adj=multipletests(pvals=np.array(spPval[sp]), alpha=0.05, method="fdr_bh")
    print (sp,Counter(adj[0]))
    for fam,pth,pval in zip(spFam[sp],adj[0],adj[1]):
        famEnrich[fam][sp]=(pth,pval)
        


Homsap Counter({False: 8825, True: 25})
Lytvar Counter({False: 9876, True: 45})
Lepocu Counter({False: 8826, True: 13})
Ptyfla Counter({False: 8711, True: 79})
Astrub Counter({False: 9924, True: 16})
Strpur Counter({False: 10039, True: 39})
Nemvec Counter({False: 8420, True: 38})
Parliv Counter({False: 10074, True: 60})
Limpol Counter({False: 7912, True: 20})
Cioint Counter({False: 7304, True: 9})
Apojap Counter({False: 9052, True: 42})
Sackow Counter({False: 9894, True: 58})
Auraur Counter({False: 6310, True: 55})
Linana Counter({False: 9940, True: 22})
Patmin Counter({False: 8668, True: 35})
Molocc Counter({False: 7662, True: 36})
Tricas Counter({False: 7022, True: 39})
Braflo Counter({False: 9638, True: 39})
Captel Counter({False: 9345, True: 63})
Bralan Counter({False: 10030, True: 43})
Patyes Counter({False: 9626, True: 34})
Annjap Counter({False: 9654, True: 28})
Lotgig Counter({False: 9281, True: 27})
Calmil Counter({False: 8628, True: 13})
Acapla Counter({False: 9854, True: 14}

In [11]:
HsaNames={e[0]:e[1] for e in csv.reader(open('Hsa_mart_info.txt'),delimiter='\t')}

In [30]:
Panther={e[0]:e[7].split(',') for e in csv.reader(open('/Users/fmarletaz/Dropbox/Genomes/Urchin/Annotation/Pliv_genes_master_filt.txt'),delimiter='\t')}

In [ ]:
print(Panther)

In [23]:
PlivNames={e[0]:e[1] for e in csv.reader(open('/Users/fmarletaz/Dropbox/Genomes/Urchin/Annotation/Pliv_genes_master_filt.txt'),delimiter='\t')}

In [ ]:
print(Pfam)

In [70]:
Pfam={e[0]:e[5].split(',') for e in csv.reader(open('/Users/fmarletaz/Dropbox/Genomes/Urchin/Annotation/Pliv_genes_master_filt.txt'),delimiter='\t')}

In [29]:
gPos={e[3].split('.')[0]:"{}:{}-{}".format(e[0],e[1],e[2]) for e in csv.reader(open('/Users/fmarletaz/Dropbox/Genomes/Urchin/Paracentrotus_genome/Analyses/Architecture/Pliv_aH2p.rcl.bed'),delimiter='\t')}

In [30]:
print(list(gPos.items())[0:4])

[('PL00001', 'Scaffold_3062:2905-5121'), ('PL00002', 'Scaffold_3062:1081-1618'), ('PL00003', 'Scaffold_1901:19325-25241'), ('PL00004', 'Scaffold_1901:7664-12225')]


In [82]:
enrList=[]
for fid,gbs in Fams.items():
    cbs=dict((sp,len(gs)) for sp,gs in gbs.items())
    pvl=famEnrich[fid]
    plnm=[PlivNames.get(g,'-') for g in gbs.get('Parliv',[])]
    plnm=[d for d in plnm if not d.startswith('unchar') and not d=='-']
        #print(plnm)
    hsnm=[HsaNames[g] for g in gbs.get('Homsap',[]) if g in HsaNames]
    pfam=set(list(sum([Pfam[g] for g in gbs.get('Parliv',[]) if g in Pfam],[])))
    #print(pfam)
    panther=set(list(sum([Panther[g] for g in gbs.get('Parliv',[]) if g in Panther],[])))
    enriched= [(s,p[1]) for s,p in pvl.items() if p[0]==True]
    if len(enriched)==0:
        continue
    #print(enriched)
    outList=[fid,len(cbs),sum(cbs.values()),np.median(list(cbs.values())),','.join([s for s,p in enriched]),
            ','.join([str(cbs[s]) for s,p in enriched]),','.join(plnm),
             ','.join(hsnm),','.join(['\'{}\''.format(g) for g in gbs['Parliv']]),','.join(set(panther)),','.join(set(pfam)),','.join(['{:.2e}'.format(float(p)) for s,p in enriched])]
    enrList.append(outList)  
        
    

In [63]:
print(pfam)

{'-'}


In [83]:
enrichPD=pd.DataFrame(enrList, columns = ['FID', 'NbSp','NbGenes','MedGeneNb','Enriched','NbEnriched','Pliv_names','Hsa_names','Pliv_id','Panther','PFAM','pvals'])  

In [84]:
enrichPD[enrichPD.Enriched.str.contains('Parliv')].to_csv('enriched_in_Parliv.txt',sep='\t')

In [ ]:
enrichPD[enrichPD.Enriched.str.contains('Parliv')]

## Gain and losses

In [31]:
def checkFam(fam,spTd):
    fid,gbs=fam
    spset=[s for s,gbs in gbs.items() if len(gbs)>0]
    #print(spset)
    #print(phtyp.name)
    #phyloCt[phtyp.name]+=1
    #gbs[Ai]
    #spTp=copy.deepcopy(spTd)
    for leaf in spTd:
        pv='1' if leaf.name in gbs else '0'
        leaf.add_features(presence=pv)
    phtyp=spTd.get_common_ancestor(spset) if len(spset)>1 else spT.search_nodes(name=spset[0])
    ndesc=len([l.name for l in phtyp.get_leaves()])
    lost=[node.name  for node in phtyp.get_monophyletic(values=['0'], target_attr="presence")]
    #print spTd.get_ascii(attributes=["name","presence"], show_internal=True)
    #for node in spTd.get_monophyletic(values=['0'], target_attr="presence"):
    #    print node.name
    #    print node.get_ascii(attributes=["presence", "name"], show_internal=False)
    return fid,len(gbs.keys()),ndesc,phtyp.name,lost



In [16]:
def checkFamCounts(fam,spTd):
    fid,cbs=fam
    spset=[s for s,n in cbs.items() if n>0]
    if len(spset)>1:
        phtyp=spT.get_common_ancestor(spset)
    else:
        phtyp=[l for l in spT.get_leaves() if l.name==spset[0]][0]
    #print phtyp.name
    #phyloCt[phtyp.name]+=1
    #gbs[Ai]
    ndesc=len([l.name for l in phtyp.get_leaves()])
    for leaf in spTd:
        pv='1' if leaf.name in spset else '0'
        leaf.add_features(presence=pv)
    lost=[node.name  for node in phtyp.get_monophyletic(values=['0'], target_attr="presence")]
    #print spTd.get_ascii(attributes=["name","presence"], show_internal=True)
    #for node in spTd.get_monophyletic(values=['0'], target_attr="presence"):
    #    print node.name
    #    print node.get_ascii(attributes=["presence", "name"], show_internal=False)
    return fid,len(spset),ndesc,phtyp.name,lost




In [88]:
checkFam(list(Fams.items())[555],spT)

('OG_559', 25, 26, 'Eumetazoa', ['Auraur'])

In [89]:
print(list(Fams.items())[555])

('OG_559', defaultdict(<class 'list'>, {'Linana': ['LANA25934'], 'Bralan': ['BL04443'], 'Strpur': ['LOC592736'], 'Parliv': ['PL21635'], 'Braflo': ['BL153011'], 'Sackow': ['Sakowv30029968m'], 'Lytvar': ['L_var_26160-RA'], 'Homsap': ['ENSG00000163291'], 'Patyes': ['PY_T06293'], 'Apojap': ['BSL78_00950'], 'Limpol': ['XP_013789801', 'XP_013790183'], 'Captel': ['CapteG138960'], 'Patmin': ['PMI_028803-RA', 'PMI_014526-RA'], 'Lepocu': ['ENSLOCG00000007313'], 'Acapla': ['LOC110976640'], 'Calmil': ['SINCAMG00000004329', 'SINCAMG00000015047'], 'Lotgig': ['LotgiG144778'], 'Molocc': ['g07092'], 'Nemvec': ['NEMVEDRAFT_v1g227246'], 'Tricas': ['TC000446'], 'Dromel': ['FBgn0038256'], 'Cioint': ['ENSCING00000018091', 'ENSCING00000019528'], 'Annjap': ['LOC117103605'], 'Astrub': ['AR14388'], 'Ptyfla': ['pfl_40v0_9_20150316_1g11072']}))


In [9]:
print(Fams['OG_65'])

defaultdict(<class 'list'>, {'Xentro': ['LOC100496101', 'steap1', 'LOC100145516', 'steap2', 'steap4'], 'Letjap': ['g652', 'g3360', 'g3361', 'g888'], 'Latcha': ['ENSLACG00000017667', 'ENSLACG00000003461', 'ENSLACG00000004138'], 'Petmar': ['PMZ_0030391-RA', 'PMZ_0037405-RA', 'PMZ_0040255-RA', 'PMZ_0040256-RA', 'PMZ_0032253-RA'], 'Homsap': ['ENSG00000157214', 'ENSG00000105889', 'ENSG00000164647', 'ENSG00000127954', 'ENSG00000115107'], 'Ambrad': ['XP_032869053', 'XP_032868575', 'XP_032868590', 'XP_032868581', 'XP_032880496', 'XP_032880497', 'XP_032880499', 'XP_032869019'], 'Danrer': ['ENSDARG00000075641', 'ENSDARG00000055901', 'ENSDARG00000062887'], 'Chipun': ['Chipu0000159', 'Chipu0000161', 'Chipu0000181', 'Chipu0003586'], 'Scytor': ['Scyto0004572', 'Scyto0004573', 'Scyto0012267', 'Scyto0006159'], 'Braflo': ['BF19766', 'BF27795', 'BF02799', 'BF02847'], 'Leueri': ['LE26648', 'LE27505', 'LE08360', 'LE27593'], 'Lepocu': ['ENSLOCG00000003240', 'ENSLOCG00000010512', 'ENSLOCG00000010454', 'ENSL

In [32]:
origins=defaultdict(int)
losses=defaultdict(int)
idGainLoss={}
for fam in Fams.items():
    fid,nspec,ndesc,oritax,lost=checkFam(fam,spT)
    idGainLoss[fid]=(nspec,ndesc,oritax,lost)
    origins[oritax]+=1
    for tax in lost:
        losses[tax]+=1
        

In [ ]:
for f,e in idGainLoss.items():
    if e[2]==''

In [33]:
spTi=copy.deepcopy(spT)
for node in spTi.traverse("postorder"):
    #print node.name,origins[node.name],losses[node.name]
    #node.add_features(origins=origins[node.name],losses=losses[node.name])
    node.name="{0}_{1}_{2}_{3}".format(node.name,origins[node.name],losses[node.name],totdups[node.name])

In [59]:
spTg=copy.deepcopy(spT)
for node in spTg.traverse("postorder"):
    #print node.name,origins[node.name],losses[node.name]
    #node.add_features(origins=origins[node.name],losses=losses[node.name])
    if not node.is_leaf():
        #node.name="{0}_{1}_{2}_{3}".format(node.name,origins[node.name],losses[node.name],totdups[node.name])
        node.name="{0}".format(totdups[node.name])

In [60]:
print(spTg.write(format=1))

((Auraur:1,Nemvec:1)498:1,((((((Homsap:1,Lepocu:1)273:1,Calmil:1)3404:1,(Cioint:1,Molocc:1)1072:1)246:1,(Braflo:1,Bralan:1)1731:1)288:1,((Ptyfla:1,Sackow:1)1167:1,(Annjap:1,((Acapla:1,Patmin:1)1043:1,(Apojap:1,(Parliv:1,(Lytvar:1,Strpur:1)348:1)2053:1)316:1)851:1)930:1)687:1)1145:1,(((Dromel:1,Tricas:1)0:1,Limpol:1)190:1,((Lotgig:1,Patyes:1)0:1,(Captel:1,Linana:1)0:1)410:1)399:1)3748:1);


In [ ]:
print(spTi.get_ascii(show_internal=True))

In [91]:
igl=[[s,len(sum(list(Fams[s].values()),[])),v[0],v[1],v[2],','.join(v[3])] for s,v in idGainLoss.items()]

In [92]:
fidGL=pd.DataFrame(igl, columns = ['FID', 'NbGenes','NbSpecies','NbDesc','Strata','Losses'])  

In [93]:
plnp={fid:','.join([PlivNames[g] for g in gbs.get('Parliv',[]) if g in PlivNames]) for fid,gbs in Fams.items()}

In [94]:
plid={fid:','.join([g for g in gbs.get('Parliv',[])]) for fid,gbs in Fams.items()}

In [95]:
hsnm={fid:','.join([HsaNames.get(g,'-') for g in gbs.get('Homsap',[])]) for fid,gbs in Fams.items()}

In [179]:
panth={fid:','.join(set(sum([Panther.get(g,['-']) for g in gbs.get('Parliv',[])],[]))) for fid,gbs in Fams.items()}

In [178]:
sum([Panther.get('PL40550',['-']) for g in gbs.get('Parliv',[])],[])

['protein wnt-2']

In [180]:
pfam={fid:','.join(set(sum([Pfam.get(g,['-']) for g in gbs.get('Parliv',[])],[]))) for fid,gbs in Fams.items()}

In [96]:
len(sum(list(Fams['OG_1'].values()),[]))

36

In [ ]:
print(pfam)

In [97]:
fidGL['Pliv_names']=fidGL['FID'].map(plnp) 

In [98]:
fidGL['Hsa_names']=fidGL['FID'].map(hsnm) 

In [99]:
fidGL['Pliv_ids']=fidGL['FID'].map(plid) 

In [167]:
fidGL['Duplications']=fidGL['FID'].map(dupNodes) 

In [181]:
fidGL['Panther']=fidGL['FID'].map(panth) 

In [182]:
fidGL['PFAM']=fidGL['FID'].map(pfam) 

In [184]:
fidGL[fidGL.Losses.str.contains('Echinoidea')]#.to_csv('lost_in_osteo.txt',sep='\t')

,FID,NbGenes,NbSpecies,NbDesc,Strata,Losses,Pliv_names,Hsa_names,Pliv_ids,Duplications,Panther,PFAM
39,OG_40,11,6,24,Bilateria,"Tunicata,Cephalochordata,Ptyfla,Annjap,Asteroi...",,"TCIRG1,ATP6V0A2",,"Lepocu,Vertebrata",,
43,OG_44,16,15,24,Bilateria,"Homsap,Tunicata,Echinoidea,Ecdysozoa",,,,Bilateria,,
81,OG_82,3,3,17,Deuterostomia,"Vertebrata,Cioint,Cephalochordata,Sackow,Annja...",,,,NaN,,
123,OG_124,3,3,8,Echinodermata,"Acapla,Astrub,Echinoidea",,,,NaN,,
184,OG_186,28,16,24,Bilateria,"Tunicata,Asteroidea,Echinoidea",,"SCRN1,SCRN2,SCRN3",,"Captel,Limpol,Vertebrata,Bilateria,Apojap",,
...,...,...,...,...,...,...,...,...,...,...,...,...
17525,OG_18164,6,5,26,Eumetazoa,"Nemvec,Vertebrata,Bralan,Hemichordata,Annjap,A...",,,,NaN,,
17530,OG_18170,2,2,17,Deuterostomia,"Olfactores,Bralan,Hemichordata,Annjap,Asteroid...",,,,NaN,,
17531,OG_18171,3,2,26,Eumetazoa,"Nemvec,Chordata,Hemichordata,Annjap,Asteroidea...",,,,NaN,,
17798,OG_18459,2,2,17,Deuterostomia,"Vertebrata,Cioint,Cephalochordata,Hemichordata...",,,,NaN,,


In [186]:
fidGL[fidGL.Strata.str.contains('Echinodermata')]#.to_csv('novel_in_echinoidea.txt',sep='\t')

,FID,NbGenes,NbSpecies,NbDesc,Strata,Losses,Pliv_names,Hsa_names,Pliv_ids,Duplications,Panther,PFAM
34,OG_35,7,7,8,Echinodermata,Apojap,unchar-8163,,PL34457,,subfamily not named,7tm_2
123,OG_124,3,3,8,Echinodermata,"Acapla,Astrub,Echinoidea",,,,NaN,,
257,OG_260,7,6,8,Echinodermata,"Patmin,Parliv",,,,Echinodermata,,
297,OG_300,3,3,8,Echinodermata,"Patmin,Echinozoa",,,,NaN,,
624,OG_629,6,3,8,Echinodermata,"Patmin,Echinozoa",,,,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...
17746,OG_18403,2,2,8,Echinodermata,"Acapla,Astrub,Echinozoa",,,,NaN,,
17762,OG_18420,2,2,8,Echinodermata,"Acapla,Astrub,Echinozoa",,,,NaN,,
17764,OG_18422,2,2,8,Echinodermata,"Patmin,Astrub,Echinozoa",,,,NaN,,
17765,OG_18423,2,2,8,Echinodermata,"Asteroidea,Apojap,B",unchar-705,,PL32783,NaN,-,-


In [ ]:
fidGL[fidGL.Losses.str.contains('Osteichthyes')]#.to_csv('lost_in_osteo.txt',sep='\t')

In [187]:
fidGL.to_csv('all_families_gain_loss.txt',sep='\t')

In [ ]:
gpos=

## Duplications 

In [1]:
def decoTree(t,outtax=['Nemvec','Auraur']):
    for l in t:
        sp,gid=l.name.split('_',1)
        gid=gid.split('.')[0]
        l.add_features(species=sp,gid=gid)
    #first, reset rooting with mid-point rooting
    R = t.get_midpoint_outgroup()
    t.set_outgroup(R)
    #then root with deuterostome ancestor
    sot=[l for l in t if l.species in outtax] 
    if len(sot)>0:
        anc=t.get_common_ancestor(sot)
        if not anc==t:
            t.set_outgroup(anc)
    return t

In [2]:
def age(node,sptr):
    tsp=list(set([lf.species for lf in node]))
    if len(tsp)==1:
        return tsp[0]
    else:
        tanc=sptr.get_common_ancestor(tsp).name
        return tanc

In [5]:
rTrees={}
nclgbfl=[]
strees={}
nexcl=0
for l in open('urch_rec.trees'):
    o,ts=l.strip().split(':',1)
    og=o.split('/')[-1].rsplit('_',1)[0]
    #print(og)
    strees[og]=ts
    t=Tree(ts, format=1)
    try:
        t=decoTree(t)
        spl=list(set([l.species for l in t]))
    #print(spl)
        if len(spl)<3:
            nexcl+=1
            continue
        rTrees[og]=t
    except:
        print("troubles with {}".format(og))

In [8]:
print(nexcl,len(rTrees))

0 11274


In [36]:
dupNodes={}
totdups=defaultdict(int)
for f,st in rTrees.items():
    dupnds=[sst for sst in st.search_nodes(D='Y')]
    ages=[age(n,spT) for n in dupnds]
    for a in list(set(ages)):
        totdups[a]+=1
    dupNodes[f]=','.join(list(set(ages)))
    #print(f,dupnds,ages)

In [37]:
print(totdups)

defaultdict(<class 'int'>, {'Asteroidea': 1043, 'Patmin': 5815, 'Eumetazoa': 2452, 'Nemvec': 2275, 'Parliv': 1548, 'Echinoidea': 2053, 'Strpur': 2827, 'Limpol': 3706, 'Vertebrata': 3404, 'Spiralia': 410, 'Ecdysozoa': 190, 'Osteichthyes': 273, 'Cephalochordata': 1731, 'Apojap': 3845, 'Sackow': 1558, 'Annjap': 1542, 'Lotgig': 1285, 'Homsap': 1310, 'Bilateria': 3748, 'Ptyfla': 2242, 'Echinodermata': 930, 'Calmil': 410, 'Braflo': 939, 'Molocc': 1824, 'Captel': 2070, 'Deuterostomia': 1145, 'Acapla': 524, 'Bralan': 1466, 'Dromel': 740, 'Auraur': 1311, 'Hemichordata': 1167, 'Lytvar': 2379, 'Tunicata': 1072, 'Ambulacraria': 687, 'Cioint': 778, 'Chordata': 288, 'Olfactores': 246, 'Eleutherozoa': 851, 'Protostomia': 399, 'Lepocu': 434, 'Cnidaria': 498, 'Echinozoa': 316, 'B': 348})


In [13]:
dupNodes={}
dupEchG=defaultdict(set)
totdups=defaultdict(int)
for f,st in rTrees.items():
    #dupnds=[sst for sst in st.search_nodes(D='Y')]
    for n in st.search_nodes(D='Y'):
        a=age(n,spT)
        if a=='Echinoidea':
            for l in n.search_nodes(species='Parliv'):
                dupEchG[f].add(l.name.split('_',1)[1])

    
    



In [14]:
print(len(dupEchG))

2053


In [15]:
print(np.median([len(g) for f,g in dupEchG.items()]))

2.0


In [ ]:
with open('dups_Echinoidea')for g in dupEchG:
    

In [13]:
print(list(dupNodes.items())[0:5])

[('OG_1000', 'Valvatida,Patmin'), ('OG_10011', 'Nemvec,Eumetazoa'), ('OG_10015', 'Echinoidea,Parliv,Strpur'), ('OG_1001', 'Spiralia,Ecdysozoa,Limpol,Vertebrata,Patmin,Cephalochordata,Osteichthyes'), ('OG_10027', 'Apojap,Sackow,Limpol')]


In [16]:
i=0
for f,st in rTrees.items():
    pliv=[sst.name for sst in st.search_nodes(species='Parliv')]
    if len(pliv)<2: continue
    for g1,g2 in list(combinations(pliv,2)):
        anc=age(st.get_common_ancestor([g1,g2]),spT)
        gi1,gi2=g1.split('_',1)[1],g2.split('_',1)[1]
        m1,m2=gene2mod.get(gi1,'-'),gene2mod.get(gi2,'-')
        print(f,g1,g2,PlivNames.get(gi1,'-'),PlivNames.get(gi2,'-'),m1,m2,moddesc.get(m1,'-'),moddesc.get(m2,'-'),anc)
        
    i+=1
    #ages=[age(n,spT) for n in dupnds]
    #for a in list(set(ages)):
    #    totdups[a]+=1
    #dupNodes[f]=','.join(list(set(ages)))
    #print(f,dupnds,ages)
    if i==20: break

NameError: name 'gene2mod' is not defined

In [127]:
print(rTrees['OG_10015'].get_ascii(attributes=["name","D"], show_internal=True))


                     /-Lytvar_L_var_25220-RA, N
                /70, N
               |    |      /-Strpur_LOC589280, N
           /66, N    \n11, Y
          |    |           \-Strpur_LOC115923326, N
          |    |
          |     \-Parliv_PL20127, N
          |
      /95, Y              /-Parliv_PL19596, N
     |    |          /67, Y
     |    |         |     \-Parliv_PL20177, N
     |    |     /17, Y
     |    |    |    |      /-Parliv_PL37751, N
     |    |    |     \100, Y
-n73, N    \29, N          \-Parliv_PL19959, N
     |         |
     |         |     /-Lytvar_L_var_25221-RA, N
     |          \49, N
     |               \-Strpur_LOC763046, N
     |
      \-Auraur_scaffold1886_g2, N


In [17]:
i=0
gDupAge={}
for f,st in rTrees.items():
    #print(f)
    pliv=[sst.name for sst in st.search_nodes(species='Parliv')]
    for l in st.search_nodes(species="Parliv"):
        gid=l.name.split('_',1)[1]
        gda='PreMet'
        #gda=['PreMet']
        for n in l.iter_ancestors():
            if n.D=='Y':
                gda=age(n,spT)
                break
        gDupAge[gid]=gda
    i+=1
    #ages=[age(n,spT) for n in dupnds]
    #for a in list(set(ages)):
    #    totdups[a]+=1
    #dupNodes[f]=','.join(list(set(ages)))
    #print(f,dupnds,ages)
    #if i==3: break    
#print(Counter(gDupAge.values()))

In [18]:
print(Counter(gDupAge.values()))

Counter({'Parliv': 7594, 'Echinoidea': 4374, 'PreMet': 3097, 'Bilateria': 1949, 'Eumetazoa': 872, 'Eleutherozoa': 648, 'Echinodermata': 633, 'Deuterostomia': 555, 'Ambulacraria': 415, 'Echinozoa': 255})


In [25]:
selDupCl=['PreMet','Eumetazoa','Bilateria','Deuterostomia','Echinodermata','Ambulacraria','Eleutherozoa','Echinozoa','Echinoidea','Parliv']

In [26]:
i=0
gDupAge={}
fgd={}
for f,st in rTrees.items():
    #print(f)
    pliv=[sst.name for sst in st.search_nodes(species='Parliv')]
    for l in st.search_nodes(species="Parliv"):
        gid=l.name.split('_',1)[1]
        #gda='PreMet'
        gda=['PreMet']
        for n in l.iter_ancestors():
            if n.D=='Y':
                #print(n.name,n.D,age(n,spT))
                gda.append(age(n,spT))
                
        gDupAge[gid]=[1 if c in gda else 0 for c in selDupCl]
        fgd[gid]=f 
    i+=1
    #ages=[age(n,spT) for n in dupnds]
    #for a in list(set(ages)):
    #    totdups[a]+=1
    #dupNodes[f]=','.join(list(set(ages)))
    #print(f,dupnds,ages)
    #if i==3: break    
#print(Counter(gDupAge.values()))

In [27]:
gbd=defaultdict(list)
for g in gDupAge:
    d=gDupAge[g]
    if d[-1]==0 and d[-2]==1:
        if not g in fgd: continue
        gbd[fgd[g]].append(g)
        #print(fgd[g],g,gPos.get(g,''),d)


In [ ]:
dupEchG

In [58]:
dists=[]
nscafs=[]
ngsc,ngt=0,0
i=0
for f in dupEchG:
    if len(dupEchG[f])>=2:
        i+=1
        pos=[gPos[g] for g in dupEchG[f]]
        scaf=[s.split(':')[0] for s in pos]
        for s,c in Counter(scaf).items():
            if c>=2:
                ngsc+=1 
        ngt+=sum(Counter(scaf).values())
        stp=[int(s.split(':')[1].split('-')[0]) for s in pos]
        if len(set(scaf))==1:
            d=max([abs(m2-m1) for m1,m2 in combinations(stp,2)])
            dists.append(d)
        nscafs.append(len(set(scaf)))
        #print(f,gbd[f],pos,stp,d)
        #if i==30: break

In [ ]:
dupNodes={}
totdups=defaultdict(int)
for f,st in rTrees.items():
    dupnds=[sst for sst in st.search_nodes(D='Y')]
    ages=[age(n,spT) for n in dupnds]
    for a in list(set(ages)):
        totdups[a]+=1
    dupNodes[f]=','.join(list(set(ages)))
    #print(f,dupnds,ages)

In [55]:
print(951)

1136 3621


In [59]:
print(Counter(nscafs),len(gbd))

Counter({1: 951, 2: 274, 3: 112, 4: 61, 5: 43, 6: 25, 8: 24, 7: 19, 11: 13, 12: 11, 9: 11, 10: 6, 17: 4, 13: 3, 16: 3, 26: 2, 24: 2, 21: 2, 15: 2, 18: 1, 14: 1, 46: 1, 33: 1, 41: 1, 30: 1, 31: 1}) 1939


In [61]:
print(len([d for d in dists if d<100000]))

703


In [62]:
print(703/1939)

0.36255801959773076


In [49]:
with open('dups_classif.tsv','w') as out:
    out.write('fid\tgid\t{}\n'.format('\t'.join(selDupCl)))
    for g in gDupAge:
        out.write('{}\t{}\t{}\t{}\n'.format(fgd[g],g,'\t'.join(map(str,gDupAge[g])),gPos.get(g,'')))
        
        
        
    

In [83]:
print(list(gDupAge.items())[0:5])

[('PL09166', [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('PL20127', [1, 0, 0, 0, 0, 0, 0, 0, 1, 0]), ('PL19596', [1, 0, 0, 0, 0, 0, 0, 0, 1, 1]), ('PL20177', [1, 0, 0, 0, 0, 0, 0, 0, 1, 1]), ('PL37751', [1, 0, 0, 0, 0, 0, 0, 0, 1, 1])]


## Importing WGCNA information

In [33]:
gene2mod={}
mod2gene={}
for f in glob("/Users/fmarletaz/Dropbox/Genomes/Urchin/Annotation/Pliv_WGCNA_log2fpkm1/test2.2_filtered_gene_models_less_clusters/Genes_in_Modules/modules_genes/*.txt"):
    mod=f.split('/')[-1].split('.')[0]
    genes=[]
    for l in open(f):
        gene2mod[l.strip()]=mod
        genes.append(l.strip())
    mod2gene[mod]=genes  
    

In [34]:
moddesc={}
for l in open("/Users/fmarletaz/Dropbox/Genomes/Urchin/Annotation/Pliv_WGCNA_log2fpkm1/test2.2_filtered_gene_models_less_clusters/cluster_desc.txt"):
    mod,desc=l.strip().split()
    moddesc[mod]=desc

In [35]:
moddesc['maroon']

'larva_adult_no_ovary'

In [48]:
modstra=[]
gbstr={}
for m in mod2gene:
    cnts=Counter([genephylo.get(g,'Parliv') for g in mod2gene[m]])
    gbstr[m]={c:cnts[c] for c in selclades}
    modstra.append([m,moddesc[m]]+[cnts[c] for c in selclades])

In [73]:
moddupa=[]
gbdpa={}
for m in mod2gene:
    cnts=Counter([gDupAge.get(g,'Parliv') for g in mod2gene[m]])
    gbdpa[m]={c:cnts[c] for c in selclades}
    moddupa.append([m,moddesc[m]]+[cnts[c] for c in selclades])

In [ ]:
moddupa

In [ ]:
modstra #['grey60', 'blastula_gastrula_pluteus_not_prism', 409, 40, 3, 1, 1, 1, 0, 9]

#### Hypergeometric test on origin (strata) age

In [ ]:
M=26054
n=tbc['Ambulacraria']
N=len()
stats.scipy.stats.hypergeom(M, n, N)

In [157]:
sum([sum(e.values()) for e in gbstr.values()])


26054

In [49]:
tbc=defaultdict(int)
for e in gbstr.values():
    for c in selclades:
        tbc[c]+=e[c]

In [53]:
hypres=[]
for m,e in gbstr.items():
    pv=[]
    #print(e)
    for c in selclades:
        hpd=stats.hypergeom(26054, tbc[c],sum(e.values()))
        pv.append(hpd.pmf(e[c]))
    hypres.append([m,moddesc[m]]+pv)
    

In [54]:
modstrati=pd.DataFrame(modstra, columns = ['mod', 'mod_desc']+selclades)  

In [55]:
modstrati.to_csv('strati_wgcna_nb.txt',sep='\t')

In [56]:
modstratihyp=pd.DataFrame(hypres, columns = ['mod', 'mod_desc']+selclades)  

In [57]:
modstratihyp.to_csv('strati_wgcna_phyp.txt',sep='\t')

#### Hypergeometric test on duplication ages

In [62]:
sum([sum(e.values()) for e in gbdpa.values()])



31313

In [74]:
tbd=defaultdict(int)
for e in gbdpa.values():
    for c in selclades:
        tbd[c]+=e[c]

In [75]:
hypresa=[]
for m,e in gbdpa.items():
    pv=[]
    #print(e)
    for c in selclades:
        hpd=stats.hypergeom(31313, tbd[c],sum(e.values()))
        pv.append(hpd.pmf(e[c]))
    hypresa.append([m,moddesc[m]]+pv)

In [76]:
selclades=['Eumetazoa','Bilateria','Deuterostomia','Ambulacraria', 'Echinodermata','Eleutherozoa','Echinozoa','Echinoidea','Parliv']

In [77]:
tbd

defaultdict(int,
            {'Eumetazoa': 850,
             'Bilateria': 1938,
             'Deuterostomia': 543,
             'Ambulacraria': 404,
             'Echinodermata': 613,
             'Eleutherozoa': 637,
             'Echinozoa': 238,
             'Echinoidea': 4276,
             'Parliv': 18748})

In [78]:
modduplage=pd.DataFrame(moddupa, columns = ['mod', 'mod_desc']+selclades)

In [79]:
modduplage.to_csv('duplage_wgcna_nb.txt',sep='\t')

In [80]:
modduplahyp=pd.DataFrame(hypresa, columns = ['mod', 'mod_desc']+selclades)

In [81]:
modduplahyp.to_csv('duplage_wgcna_phyp.txt',sep='\t')

In [66]:
igl=[[g,PlivNames.get(g,'-'),genephylo.get(g,'Parliv'),m,moddesc[m]] for g,m in gene2mod.items()]

NameError: name 'PlivNames' is not defined

In [65]:
gstrmod=pd.DataFrame(igl, columns = ['gid', 'gname','Strata','Module','ModuleDesc'])  

NameError: name 'igl' is not defined

In [64]:
#gstrmod[gstrmod.Strata.str.contains('Echinozoa')]#.to_csv('novel_in_echinoidea.txt',sep='\t')
gstrmod[gstrmod.Module==('lightyellow')]#.to_csv('novel_in_echinoidea.txt',sep='\t')

NameError: name 'gstrmod' is not defined

In [292]:
gstrmod.to_csv('genes_wgcna_strata.txt',sep='\t')

In [ ]:
for col in cols:
    col_zscore = col + '_zscore'
    df[col_zscore] = (df[col] - df[col].mean())/df[col].std(ddof=0)

In [ ]:
modstratihyp

## Output